In [11]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [12]:
%store -r train
%store -r null_vals

In [13]:
print(train.shape)

(1448, 195)


In [15]:
y = train.log_SalePrice
X = train.drop(['SalePrice', 'Id', 'log_SalePrice'], axis=1)
print(X.shape)
print(y.shape)

(1448, 192)
(1448,)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,
                                                    random_state=42)

In [17]:
ols = LinearRegression()
ols.fit(X_train, y_train)
print("R^2 for train set: %f" %ols.score(X_train, y_train))
print('-'*50)
print("R^2 for test  set: %f" %ols.score(X_test, y_test))

R^2 for train set: 0.947442
--------------------------------------------------
R^2 for test  set: 0.903612


In [19]:
predictions = ols.predict(X_test)
from sklearn.metrics import mean_squared_error
print ('RMSE is: \n', mean_squared_error(y_test, predictions))

RMSE is: 
 0.015166873231451286
